## Building Blocks

_Optimization is the route to all evil_

_Getting right first and fast then_ by D. Knuth. AKA "Get it right first, then make it fast".

How do we do this? 

Useful links:
- We'll follow the concise [Software Carpentry Testing Tutorial](http://carpentries-incubator.github.io/python-testing/) authored by [Dr. Katy Huff](http://katyhuff.github.io). 
- Also [this Dr. Katy Huff](https://www.energy.gov/ne/person/dr-kathryn-huff).
- [The First Notebook War](https://yihui.org/en/2018/09/notebook-war/)

````{note}
There isn’t a clear borderline between software engineers and data analysts.
 
How would you write unit tests for data analysis? I feel it will be both tricky and unnecessary. For a function/method, if you defined it, you know what its expected output should be. For data, you often don’t know what exactly to expect in the output. For example, when you subset a dataset, how do you know the result is correct?
```R
mtcars2 = dplyr::filter(mtcars, hp > 100)
```
That is probably not something you, as a data analyst, need to worry about. It is the responsibility of the package author (the software engineer) to write enough unit tests in the package that you are using.

On the other hand, data analysts often do tests in an informal way, too. As they explore the data, they may draw plots or create summary tables, in which they may be able to discover problems (e.g., wrong categories, outliers, and so on). Notebooks are great for these inline output elements, from which you can make quick discoveries.
````

### Simple motivation: chaotic systems and numerical precision

We are going to play with the notebook `simple-numerical-chaos.ipynb`, [notebook](simple-numerical-chaos.ipynb) that you can find in this same directoy. Consider for example the following operation

In [1]:
a, b, c = 1.0, 1e-16, 1e-16
print(f"(a + b) + c = {(a + b) + c}")
print(f"a + (b + c) = {a + (b + c)}")

(a + b) + c = 1.0
a + (b + c) = 1.0000000000000002


The problem here is caused by rounding of floating point numbers. A good reference for this is included in [What Every Computer Scientist Should Know About Floating-Point Arithmetic](https://docs.oracle.com/cd/E19957-01/806-3568/ncg_goldberg.html).

In Python, we can use a different standard for floting point numbers with the module [decimal](https://docs.python.org/3/library/decimal.html). This is particularly useful for real world cases where small number operations may be critical, for example when making tons of bank transactions. 

In [3]:
def f1(x): return r*x*(1-x)
def f2(x): return r*x - r*x**2

r = 3.9
x = 0.8
print('f1:', f1(x))
print('f2:', f2(x))

print('difference:', (f1(x)-f2(x)))

f1: 0.6239999999999999
f2: 0.6239999999999997
difference: 2.220446049250313e-16


Now, the decimal digits of the difference are just garbage: eirher `f1(x)` or `f2(x)` have no information after the last digit. 

Now, this raises the question about what does it mean to get the _right answer_ from our code and _what does it mean to be reproducible_
in scientific computing.

This short example help us to undersrand what is important in the context of computational 

```{note}
Scientist, studetents, we are always doing test of our code and our methods. We do this in a subtle way, by printing an output, making a plot, etc. All these are quite similar to unit tests. 

Exploratory data analysis

Testing also help us for creating a to do list when we make significant changes in the code (Fernando's example from numerics to numpy). 
```

### Simple testing

There is a functin in Python that help us to see how much of our current code is under testing right now. 

### Writing a test suite

Super simple: pytest just finds all the files that start with `test_` and run them. These need to be argumentless and return a bolean variable.

Notice that the output of pytest is quite inteligent: it tells you where the error has been induced and also prints the values of the variables that induce the error. 

Forcing to write test force you to write good code. 

`try`/`except` vs `if` statement.

## Different filosofies for testing

- Defensive programming: errors will happen and we need to be ready for them.
- Test-Driven development

## Types of tests

- Assertions and Exceptions
- Unit Test
- Regression Tests
- Integration Tests

### Assestions 

The `assert` statement in Python just evaluates when some given condition is true or false. If False, it interrupst the exectution of the code. 

In [2]:
assert 1+1 == 2, "One plus one is not two."

As you can see from the previous example, you can also add a small text description for the error induced. in this way, assertion statements are very simple to write and evaluate. 

As you can imagine from the discussion in the previous section, we need to be careful at the moment of comparing objects in Python. For example, for float types we have

In [4]:
assert 0.1 + 0.2 == 0.3

AssertionError: 

The problem here is induced by floating point aritmethics in our code. There are different ways of doing this properly:
- `import math`, `math.isclose()`
- `import numpy`, `np.isclose()` or `np.allclose()` for lists. 
- `import pytest`, `pytest.approx()`
- `from numpy.testing import assert_allclose`, `assert_allclose()`.

All this are valid options at the moment of comparing floting point numbers. 

In [6]:
import pytest
assert 0.1 + 0.2 == pytest.approx(0.3)

Ussually assertion statements go inside a functions or definitions an help us to keep the correctness of the code. In pair programming, it is the role of the observer to think in cases where the code may not work and think about simple assertion statements that will help prevent those errors. 

Importance between returning a bool or an assertion.

In [1]:
import numpy as np
np.isclose?

Signature: np.isclose(a, b, rtol=1e-05, atol=1e-08, equal_nan=False)
Docstring:
Returns a boolean array where two arrays are element-wise equal within a
tolerance.

The tolerance values are positive, typically very small numbers.  The
relative difference (`rtol` * abs(`b`)) and the absolute difference
`atol` are added together to compare against the absolute difference
between `a` and `b`.

.. warning:: The default `atol` is not appropriate for comparing numbers
             that are much smaller than one (see Notes).

Parameters
----------
a, b : array_like
    Input arrays to compare.
rtol : float
    The relative tolerance parameter (see Notes).
atol : float
    The absolute tolerance parameter (see Notes).
equal_nan : bool
    Whether to compare NaN's as equal.  If True, NaN's in `a` will be
    considered equal to NaN's in `b` in the output array.

Returns
-------
y : array_like
    Returns a boolean array of where `a` and `b` are equal within the
    given tolerance. If both `a` 

### Exceptions

Different kinds of errors that occur as we write code include syntax, runtime and semantic errors. Specially for runtime errors, Python give us a clue about what kind or error may happened during the execution of our code. For example,

In [7]:
1 / 0

ZeroDivisionError: division by zero

In [8]:
my_dict = {'a':1, 'b':2}
my_dict['c']

KeyError: 'c'

In [9]:
my_dict + {'c':3}

TypeError: unsupported operand type(s) for +: 'dict' and 'dict'

There are many more different kind of built-in exceptions in Python. You can find some more examples in this [link](https://docs.python.org/3/library/exceptions.html). A general `RuntimeError` is raised when the detected error doesn't fall in any of the other categories. 

There are different ways of dealing with runtime errors in Python, there include the 
- `try`...`except` clause
- `raise` statement

In [20]:
def division(numerator, denominator):
    try:
        return numerator / denominator
    except ZeroDivisionError:
        return 0

In [17]:
division(1,1)

1.0

In [19]:
division(1,0)

0

Now, at the moment of raising an error we would like to print a meaningful message. We can do this 

In [26]:
def division(numerator, denominator):
    try:
        return numerator / denominator
    except ZeroDivisionError:
        raise ZeroDivisionError("You cannot divide by denominator={}".format(denominator))

In [27]:
division(1,0)

ZeroDivisionError: You cannot divide by denominator=0

If you already know what may be causing an error in your code, you can avoind the use of the `try / excepr` statement and directly raise an exception when certain critical condition happens: 

In [57]:
def division(numerator, denominator):
    if denominator == pytest.approx(0.0):
        raise ZeroDivisionError("You cannot divide by denominator={}".format(denominator))
    return numerator / denominator

In [30]:
division(1,0)

ZeroDivisionError: You cannot divide by denominator=0

Something cool about exceptions is that their are classes and Python allow us to create new assertion errors. 

In [34]:
class LightSpeedBound(Exception):
    """
    Defines a new exception error of my preference.
    """
    pass

def lorentz_factor(v, c=299_792_458):
    if v > c:
        raise LightSpeedBound("The current velocity v={} cannot exceed the speed of light".format(v))
    return 1 / (1 - v**2/c**2) ** 0.5

In [39]:
lorentz_factor(300_000_000)

LightSpeedBound: The current velocity v=300000000 cannot exceed the speed of light

````{note}
Currently Python supports type hinting at the moment of defining new functions. Altught these are hinds and not something will be required for the function, being explicit about the input and output types helps having a more readable and accurate code
```python
def division(numerator:float, denominator:float) -> float:
    return numerator / demoninator
```
````

check on `myPy`

### Unit Tests

In previous section we were discussion about the importance of writting clean and modular code. Having small functions that perfom very specific task help us also to desing pipelines for testing those small units of code. That is the purpose of unit tests, to individually test the functions in our code. 

The way of writing unit tests consist in defining function that will return an `assert` statement testing whenever the output matches the _true_ answer.

In [61]:
import numpy as np

def division(numerator, denominator):
    if denominator == pytest.approx(0.0):
        raise ZeroDivisionError("You cannot divide by denominator={}".format(denominator))
    return numerator / denominator

def test_float_division():
    assert np.isclose(division(2.0,0.5), 4.0)

In [62]:
test_float_division()

The next step is to scalate this! Having more than one test for function that can evaluate different cases (eg, different types) and then extent to all the functions in your code. 

## Regression test

I think for this to make sense we need a good CI system. Maybe we can do an example with this after the CI section? 